### 0. Create TEXTANALYSIS User and grant rights following https://blogs.sap.com/2020/07/27/hands-on-tutorial-automated-predictive-apl-in-sap-hana-cloud/

### 0. Imports

In [1]:
import pandas as pd
from datetime import date, datetime, timedelta
import os

from hdbcli import dbapi
from hana_ml import dataframe
from hana_ml.dataframe import create_dataframe_from_pandas
from hana_ml.algorithms.pal.partition import train_test_val_split

from hana_ml.text.tm import tf_analysis
from hana_ml.text.tm import get_related_doc, get_related_term, get_relevant_doc, get_relevant_term, get_suggested_term, text_classification

pd.set_option('display.max_colwidth', None)
pd.set_option("display.colheader_justify","left")

### 1. HANA-ML Connection

In [4]:
# Instantiate connection object
conn = dataframe.ConnectionContext(address = 'XXX.hana.canary-eu10.hanacloud.ondemand.com',
                                   port = 443, 
                                   user = 'XXX', 
                                   password = 'XXX', 
                                   encrypt = 'true',
                                   sslValidateCertificate = 'false' 
                                  )

# Send basic SELECT statement and display the result
sql = 'SELECT 12345 FROM DUMMY'
df_remote = conn.sql(sql)
print(df_remote.collect())

# Set up TEXTANALYSIS User, Add rights to execute PAL
# cursor = conn.connection.cursor()
# cursor.execute('CREATE USER TEXTANALYSIS Password "Textan123" SET USERGROUP DEFAULT;')
# cursor.execute('ALTER USER TEXTANALYSIS DISABLE PASSWORD LIFETIME;')
# cursor.execute('GRANT "AFL__SYS_AFL_AFLPAL_EXECUTE_WITH_GRANT_OPTION" TO TEXTANALYSIS')

# Close connection
# conn.close()

   12345
0  12345


### 2. Acquire source data

In [5]:
df_tm_source = dataframe.DataFrame(conn,'select "CONTENT", "CATEGORY", "MFR_NAME", "MAKETXT","MODELTXT","YEARTXT","CITY","STATE" FROM COMPLAINTS').filter('MAKETXT = \'FORD\'').drop_duplicates(['CONTENT']).add_id('ID').cast('ID', 'NVARCHAR (5000)')
train, test, valid = train_test_val_split(data=df_tm_source, training_percentage = 0.09, validation_percentage = 0.8, testing_percentage = 0.11, random_seed = 41)
train.count(),valid.count(), test.count()

(10240, 91025, 12516)

In [6]:
test.select(['ID', 'MFR_NAME', 'CONTENT', 'CATEGORY']).head(10).collect()

,ID,MFR_NAME,CONTENT,CATEGORY
0,1,FORD MOTOR COMPANY,"PARKED VEHICLE AT AROUND 7:20 AM AND AT AROUND 8:30 AM THE VEHICLE CAUGHT ON FIRE. THE FIRE WAS COMING FROM THE ENGINE COMPARTMENT ON THE DRIVER'S SIDE. ONE WEEK PRIOR TO THE FIRE THE CRUISE CONTROL STOPPED WORKING. AND THE BRAKE LIGHT CAME ON THE DASHBOARD. A COUPLE OF DAYS BEFORE THE FIRE A FUSE BLEW OUT\, AND WAS REPLACED ONE TIME. THE VEHICLE WOULD NOT GO INTO PARK. STEREO AND SPEAKERS WERE INSTALLED EARLY IN 1999 PROFESSIONALLY. THE FIRE DID NOT SPREAD TO THE HOUSE. WILL MAIL 35 MM PICTURES TO BRUCE YORK.*AK",ELECTRICAL SYSTEM
1,16,FORD MOTOR COMPANY,"VEHICLE WAS SERVICED ON 15-JAN-2002 FOR RECALL CAMPAIGN 01V390000/ENGINE COOLING FAN BEARING. AFTER VEHICLE WAS SERVICED FOR RECALL AND DRIVEN 35 MPH\, ENGINE OVERHEATED AND VEHICLE STALLED. DEALERSHIP EXAMINED VEHICLE AND DETERMINED ENGINE COOLING FAN HAS FAILED AND REPAIRS WERE BEING MADE TO CORRECT THE PROBLEM. *AK *TT",ENGINE AND ENGINE COOLING:COOLING SYSTEM:FAN
2,29,FORD MOTOR COMPANY,A BURGLAR WAS ABLE TO INSERT A SCREW DRIVER BETWEEN THE DRIVER'S SIDE DOOR HANDLE MODULE AND THE OUTER WALL OF THE DOOR AND REMOVE THE HANDLE MODULE AND OPEN THE DOOR AND STEAL VARIOUS ITEMS IN THE VEHICLE WITHOUT SETTING OFF THE ALARM.*AK,LATCHES/LOCKS/LINKAGES:DOORS:LATCH
3,39,FORD MOTOR COMPANY,BRAKE FLUID CYLINDER FAILED WHICH CAUSED THE VEHICLE TO CATCH ON FIRE. *BF *NM VEHICLE PARKED,"SERVICE BRAKES\, HYDRAULIC"
4,48,FORD MOTOR COMPANY,"1998 FORD F-150; FIRE UNDER THE HOOD. TRUCK WAS PARKED IN MY DRIVEWAY WITH THE IGNITION TURNED OFF. FIRE STARTED ON THE DRIVERS SIDE NEXT TO THE FIREWALL AREA. I RECEIVED A RECALL NOTICE ON THE CRUISE CONTROL SENSOR\, BUT IT SAID PARTS WERE NOT AVAILABLE TO REPAIR UNTIL AFTER FEBRUARY 2006. FIRE MARSHALL OF CYPRESS CREEK VOLUNTEER FIRE DEPT WAS AT THE SEEN AND WROTE A REPORT. *NM",VEHICLE SPEED CONTROL:CRUISE CONTROL
5,49,FORD MOTOR COMPANY,"FRONTAL AIR BAGS DEPLOYED WHEN VEHICLE INVOLVED IN A CRASH. THEN\, DRIVER'S SIDE AIR BAG ALSO DEPLOYED AND RIPPED/TORE. WHEN AIR BAG TORE\, ELEMENTS IN AIR BAG CAUGHT ON FIRE\, AND CONSUMER WAS THROWN INTO STEERING COLUMN. DRIVER SUFFERED ABDOMINAL/ RIB CAGE \, AND ARM INJURIES WHEN HE TRIED TO STOP HIMSELF FROM GOING INTO STEERING COLUMN. PLEASE PROVIDE ANY ADDITIONAL INFORMATION/ATTACHMENTS. *AK",AIR BAGS:SIDE/WINDOW
6,55,FORD MOTOR COMPANY,"TREAD ON TIRE CAME APART. IT'S A FIRESTONE TIRE INSURED SPK W/DEARLERSHIP. THEY WANT HIM TO MAIL THE TIRE INTO THEM. INSURED HAS MADE AN APPT ALRADY W/CLAIMS FIRESTONE RADIAL ATX\, P235/75/R15. (CLAIM#53-C596-249",TIRES:TREAD/BELT
7,87,FORD MOTOR COMPANY,"WHILE DRIVING MY 2004 FORD EXPEDITION AT ~65 MPH THE CAR IN FRONT OF ME SLOWED ABRUPTLY. UPON PRESSING THE BRAKE PEDAL\, THE PEDAL FELL TO THE FLOOR AND THE BRAKES FAILED TO SLOW THE VEHICLE. I SWERVED TO AVOID HITTING OTHER VEHICLE. I REMOVED MY FOOT FROM THE BRAKE AND THEN PRESSED THE BRAKE AGAIN AND THE BRAKE BEGAN TO OPERATE NORMALLY ONCE AGAIN. CAR CURRENTLY AT THE DEALER ALTHOUGH CAUSE HAS NOT BEEN FOUND. 1 WEEK PRIOR I HAD THE BRAKES INSPECTED DUE A CONCERN WHERE THE BRAKE PEDAL WOULD SLOWLY FALL TO FLOOR WHILE ADDING AND REMOVING PRESSURE FROM THE PEDAL (WITHOUT COMPLETELY REMOVING PRESSURE) IN 5-10 MPH DOWNHILL STOP AND GO TRAFFIC. DEALER HAD TOLD ME THAT THE ENTIRE BRAKE SYSTEM HAD BEEN INSPECTED AND THAT THE BRAKES WERE FINE. *AK","SERVICE BRAKES\, HYDRAULIC:ANTILOCK"
8,90,FORD MOTOR COMPANY,"PURCHASED NEW FORD ESCAPE JANUARY\, 2005. TRANSMISSION DISENGAGED AND FAILED AUGUST\, 2005 WITH ONLY 6091 MILES ON THE ODOMETER AND IN BUSY INTERSECTION. CAR HAD TO BE TOWED TO DREW FORD DEALER...TRANSMISSION OVERHAULED. TRANSMISSION DISENGAGED ON SEPTEMBER\, 2005 WITH 6710 MILES ON THE ODOMETER. AT THIS TIME WE WERE TRAVELING 70 MPH ON FREEWAY WITH CONSIDERABLE TRAFFIC TO NEGOTIATE TO THE ROADSIDE. CAR TOWED AGAIN TO DEALER. TRANSMISSION OVERHAULED AGAIN AFTER MANUFACTURER REFUSED TO REPLACE TRANSMISSION WITH NEW ONE. WERE TOLD BY DREW FORD THAT IT WAS A DIFFERENT PART 

### 3. Train TF Model

In [7]:
tfidf= tf_analysis(train.select(['ID', 'CONTENT', 'CATEGORY']))

In [8]:
tfidf[0].describe().collect()

,column,count,unique,nulls,mean,std,min,max,median,25_percent_cont,25_percent_disc,50_percent_cont,50_percent_disc,75_percent_cont,75_percent_disc
0,TM_TERM_TF_F,15779,514,0,36.969580,450.749361,1.000000,40159.000000,2.000000,1.000000,1.000000,2.000000,2.000000,6.000000,6.000000
1,TM_TERM_IDF_F,15779,489,0,26.130173,177.380356,1.000000,6915.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,6.000000
2,TM_TERM_TF_V,15779,514,0,0.000063,0.000773,0.000002,0.068843,0.000003,0.000002,0.000002,0.000003,0.000003,0.000010,0.000010
3,TM_TERM_IDF_V,15779,489,0,8.146948,1.532914,0.392609,9.234057,9.234057,7.442297,7.442297,9.234057,9.234057,9.234057,9.234057
4,TM_TERMS,15779,15779,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
tfidf[0].head(20).collect()

,TM_TERMS,TM_TERM_TF_F,TM_TERM_IDF_F,TM_TERM_TF_V,TM_TERM_IDF_V
0,,12,6,0.000021,7.442297
1,,7,3,0.000012,8.135445
2,,9,4,0.000015,7.847763
3,,3,2,0.000005,8.540910
4,,1,1,0.000002,9.234057
5,,9,5,0.000015,7.624619
6,,1,1,0.000002,9.234057
7,,1,1,0.000002,9.234057
8,0,15,13,0.000026,6.669108
9,0.05,1,1,0.000002,9.234057


In [10]:
tfidf[1].describe().collect()

,column,count,unique,nulls,mean,std,min,max,median,25_percent_cont,25_percent_disc,50_percent_cont,50_percent_disc,75_percent_cont,75_percent_disc
0,TM_TERM_FREQUENCY,412308,43,0,1.414823,1.359601,1.0,85.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,ID,412308,10240,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TM_TERMS,412308,15779,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
tfidf[1].head(20).collect()

,ID,TM_TERMS,TM_TERM_FREQUENCY
0,10,2,1
1,10,ak,1
2,10,and,2
3,10,bed,1
4,10,cannot,1
5,10,contacted,1
6,10,dealer,1
7,10,entire,2
8,10,exists,1
9,10,fender,1


In [12]:
tfidf[2].describe().collect()

,column,count,unique,nulls,mean,std,min,max,median,25_percent_cont,25_percent_disc,50_percent_cont,50_percent_disc,75_percent_cont,75_percent_disc
0,ID,10240,10240,0,None,None,None,None,None,None,None,None,None,None,None
1,CATEGORY,10240,303,0,None,None,None,None,None,None,None,None,None,None,None


In [13]:
tfidf[2].head(20).collect()

,ID,CATEGORY
0,10,ENGINE AND ENGINE COOLING:ENGINE:GASOLINE
1,13,"SERVICE BRAKES\, HYDRAULIC:ANTILOCK"
2,17,ENGINE AND ENGINE COOLING:ENGINE:GASOLINE
3,27,POWER TRAIN:CLUTCH ASSEMBLY
4,34,"FUEL SYSTEM\, GASOLINE:DELIVERY:FUEL PUMP"
5,46,VEHICLE SPEED CONTROL:CRUISE CONTROL
6,54,SUSPENSION:FRONT:STABILIZER BAR
7,104,"SERVICE BRAKES\, HYDRAULIC:FOUNDATION COMPONENTS:DISC:ROTOR"
8,129,VEHICLE SPEED CONTROL
9,134,ENGINE AND ENGINE COOLING


### 4. Perform prediction on one document

In [14]:
test.select(['ID', 'MFR_NAME', 'CONTENT', 'CATEGORY']).head(3).collect()

,ID,MFR_NAME,CONTENT,CATEGORY
0,1,FORD MOTOR COMPANY,"PARKED VEHICLE AT AROUND 7:20 AM AND AT AROUND 8:30 AM THE VEHICLE CAUGHT ON FIRE. THE FIRE WAS COMING FROM THE ENGINE COMPARTMENT ON THE DRIVER'S SIDE. ONE WEEK PRIOR TO THE FIRE THE CRUISE CONTROL STOPPED WORKING. AND THE BRAKE LIGHT CAME ON THE DASHBOARD. A COUPLE OF DAYS BEFORE THE FIRE A FUSE BLEW OUT\, AND WAS REPLACED ONE TIME. THE VEHICLE WOULD NOT GO INTO PARK. STEREO AND SPEAKERS WERE INSTALLED EARLY IN 1999 PROFESSIONALLY. THE FIRE DID NOT SPREAD TO THE HOUSE. WILL MAIL 35 MM PICTURES TO BRUCE YORK.*AK",ELECTRICAL SYSTEM
1,16,FORD MOTOR COMPANY,"VEHICLE WAS SERVICED ON 15-JAN-2002 FOR RECALL CAMPAIGN 01V390000/ENGINE COOLING FAN BEARING. AFTER VEHICLE WAS SERVICED FOR RECALL AND DRIVEN 35 MPH\, ENGINE OVERHEATED AND VEHICLE STALLED. DEALERSHIP EXAMINED VEHICLE AND DETERMINED ENGINE COOLING FAN HAS FAILED AND REPAIRS WERE BEING MADE TO CORRECT THE PROBLEM. *AK *TT",ENGINE AND ENGINE COOLING:COOLING SYSTEM:FAN
2,29,FORD MOTOR COMPANY,A BURGLAR WAS ABLE TO INSERT A SCREW DRIVER BETWEEN THE DRIVER'S SIDE DOOR HANDLE MODULE AND THE OUTER WALL OF THE DOOR AND REMOVE THE HANDLE MODULE AND OPEN THE DOOR AND STEAL VARIOUS ITEMS IN THE VEHICLE WITHOUT SETTING OFF THE ALARM.*AK,LATCHES/LOCKS/LINKAGES:DOORS:LATCH


In [15]:
tc = text_classification(test.filter('ID = 1').select(['ID', 'CONTENT']), tfidf, thread_ratio = 1, k_nearest_neighbours = 1)
tc[0].collect()

,ID,TARGET
0,1,VEHICLE SPEED CONTROL:CRUISE CONTROL


### 5. Deep-dive into specific report

#### Investigate relevant terms in one document

In [16]:
get_relevant_term(test.filter('ID = 1').select('CONTENT'), tfidf).head(10).collect()

,ID,SCORE
0,fire,0.354257
1,bruce,0.245405
2,mm,0.245405
3,professionally,0.245405
4,speakers,0.226984
5,stereo,0.197787
6,york,0.187012
7,around,0.184712
8,spread,0.177239
9,am,0.159752


#### Find & understand documents related to one document

In [17]:
get_related_doc(test.filter('ID = 1').select('CONTENT'), tfidf).head(10).collect()

,ID,SCORE
0,113733,0.330473
1,45601,0.314277
2,64033,0.311759
3,46,0.301308
4,50690,0.301148
5,102691,0.295541
6,91386,0.295375
7,94502,0.291700
8,28770,0.286993
9,96167,0.285830


In [18]:
train.filter('ID = 113733').collect().iloc[0,1]

"VEHICLE WAS PARKED FOR 2 DAYS.  THE HORN STARTED BLOWING\\, AND CONSUMER'S WIFE LOOKED OUT THE WINDOW AND SAW FIRE COMING FROM THE CENTER OF THE HOOD.  PUT WATER ON VEHICLE FROM HOSE\\, BUT WAS NOT ENOUGH TO STOP FIRE.  CALLED 911.  BY THE TIME THE FIRE COMPANY CAME VEHICLE WAS CONSUMED BY FIRE.  ALSO\\, THE HOUSE ALMOST CAUGHT ON  FIRE.   CRUISE CONTROL QUIT WORKING ABOUT 1 OR 2 MONTHS BEFORE THE FIRE.  THERE WERE NO  OTHER PROBLEMS.  CHECKED THE FUSES\\, AND THEY WERE FINE.  ONE EVENING\\, BEFORE THE FIRE WOULD NOT COME OUT OF PARK.  FOUND THE BRAKE LIGHT SWITCH AT THE PEDAL WAS DEFECTIVE.  REPLACED THAT SWITCH.  THEN\\, FOUND A BRAKE LIGHT BULB OUT.  REPLACED THAT BULB\\, AND  VEHICLE CAME OUT OF PARK.  ABOUT A WEEK LATER THE RIGHT TURN SIGNAL BLINKED REAL FAST\\, AGAIN  THE BULB WAS REPLACED.  THIS TROUBLE WITH THE TURN SIGNAL WAS INTERMITTENT.   REAR GLASS LEAKED\\, AND  DOOR PANELS WERE CRACKED ON BOTH SIDES.  GEICO INSURANCE  SENT AN ADJUSTER TO TAKE PICTURES.  OWNER FELT CRUIS

#### Analyze one specific keyword "Cruise"

In [19]:
df_test_class = pd.DataFrame({"CONTENT":["CRUISE"]})
df_hana_test_class = create_dataframe_from_pandas(connection_context=conn,
                                                        pandas_df=df_test_class,
                                                        table_name="#TM_DATA_CLASSICATION",
                                                        force=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.89it/s]


In [20]:
get_related_term(df_hana_test_class, tfidf).head(10).collect()

,ID,SCORE
0,cruise,1.000000
1,control,0.597454
2,1fdke30g1shb03713,0.233680
3,effingham,0.233680
4,mcatee,0.233680
5,deactivation,0.233327
6,switch,0.230537
7,fire,0.211993
8,disengage,0.211546
9,distribution,0.209068


In [21]:
get_relevant_doc(df_hana_test_class, tfidf).head(10).collect()

,ID,SCORE
0,75503,0.233680
1,39932,0.181053
2,5039,0.167754
3,111140,0.157521
4,31090,0.142939
5,107827,0.141808
6,63996,0.131707
7,109691,0.128794
8,4935,0.128343
9,86406,0.124651


In [22]:
train.filter('ID = 75503').collect().iloc[0,1]

'ORIGINALLY IN DEC 2004 WHILE RETURNING FROM A TRIP TO ST LOUIS\\, MO I STARTED HAVING TROUBLE WITH MY ABS LIGHT\\, MY SPEEDOMETER HAND INTERMITTENTLY GOING BACK TO ZERO AND THE VEHICLE COMING OUT OF CRUISE CONTROL.  IT WAS ORIGINALLY THOUGHT TO BE AN ABS BRAKE PROBLEM SO THE SPEED SENSOR AND THE ABS MODULE WERE REPLACED.  THE NEXT TIME I WENT OUT ON THE ROAD FOR THE 4TH OF JULY 2005 WEEKEND\\, WE GOT AS FAR AS CHARLESTON\\, SC AND  HAD TO PUT THE RV IN THE SHOP BECAUSE THE CRUISE CONTROL AND SPEEDOMETER WERE BOTH MALFUNCTIONING; THE VEHICLE WOULD COME OUT OF CRUISE AND THE SPEEDOMETER WOULD INTERMITTENTLY GO FROM THE CORRECT SPEED TO ZERO. ALSO THE TRANSMISSION SEEMED TO BE DOWNSHIFTING AND UPSHIFTING AT WILL.  AFTER THAT WE BROUGHT THE VEHICLE BACK TO THE HOUSE AND PARKED IT ON THE DRIVEWAY.  A COUPLE WEEKS LATER I WENT OUT TO MOVE THE VEHICLE AND GIVE IT A WASH JOB AND CLEAN IT UP A BIT.  IT WOULD NOT START UPON EXAMINATION I FOUND THAT THE POWER DISTRIBUTION BOX HAD SOME MELTED REL

In [23]:
# Close connection
conn.close()